In [1]:
import numpy as np
import isodisreg 
from isodisreg import idr
from helper_functions import algo72_ensemble, log_norm_optim, crps_censored_gaussian
import properscoring as ps
import pandas as pd
import warnings
#warnings.filterwarnings('ignore')

In [2]:
# lead_time: 1, 2, 3, 4, 5 (in days)
lead_time = 1 

In [3]:
fra_data = np.loadtxt('./data/fra_lead_'+str(lead_time)+'.txt')
dim_train = 2896
dim_test = 721
dim_train2 = dim_train-dim_test
y = fra_data[:, 0]
y_train = y[0:dim_train]
y_test = y[dim_train:]

hres = fra_data[:, 1]
hres_train = hres[0:dim_train]
hres_test = hres[dim_train:]

In [4]:
# Conformal prediction

In [5]:
ens_cp = algo72_ensemble(hres_train, hres_test, y_train)
ens_cp0 = np.where(ens_cp < 0, 0, ens_cp)

print('CRPS CP ensemble:' , np.round(np.mean(ps.crps_ensemble(y_test,ens_cp)), 3))
print('CRPS zero-censored CP ensemble:', np.round(np.mean(ps.crps_ensemble(y_test,ens_cp0)),3))

CRPS CP ensemble: 0.886
CRPS zero-censored CP ensemble: 0.85


In [6]:
# EasyUQ

In [6]:
fit = idr(y_train, pd.DataFrame({'X': hres_train}))
idr_train = fit.predict()
idr_test = fit.predict(pd.DataFrame({'X': hres_test}))
print('CRPS of EasyUQ:', np.round(np.mean(idr_test.crps(y_test)), 3))

CRPS of EasyUQ: 0.732


In [ ]:
# Single Gaussian

In [7]:
sigma = log_norm_optim(y_train, hres_train)
print('CRPS of SG:',np.round(np.mean(ps.crps_gaussian(y_test, mu=hres_test, sig=sigma)), 3))
print('CRPS of censored SG:',np.round(np.mean(crps_censored_gaussian(y_test, hres_test, sigma)), 3))

CRPS of SG: 1.244
CRPS of censored SG: 1.013
